# Compare Results

This interactive notebook allows you to select completed **MATLAB** and **Python** runs and generate a comparison report.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add project root to path
project_root = Path('..').resolve()
sys.path.insert(0, str(project_root))

from source.slavv.dev.comparison import run_standalone_comparison

## Select Runs

Choose the output folders from your previous runs.

In [2]:
comparisons_dir = project_root / "comparisons"
if not comparisons_dir.exists():
    comparisons_dir.mkdir(parents=True, exist_ok=True)

# Find run folders
all_runs = sorted([d.name for d in comparisons_dir.iterdir() if d.is_dir()], reverse=True)

# Heuristic filtering
matlab_candidates = [r for r in all_runs if 'matlab' in r.lower()] + [r for r in all_runs if 'matlab' not in r.lower()]
python_candidates = [r for r in all_runs if 'python' in r.lower()] + [r for r in all_runs if 'python' not in r.lower()]

# Defaults
default_matlab = matlab_candidates[0] if matlab_candidates else None
default_python = python_candidates[0] if python_candidates else None
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
default_output_name = f"{timestamp}_comparison_report"

# Styles
section_header_style = "font-size: 14px; font-weight: bold; color: #2c3e50; margin-bottom: 5px; border-bottom: 2px solid #eaeaea; padding-bottom: 5px; width: 100%"
box_layout = widgets.Layout(
    border='1px solid #e0e0e0', 
    padding='15px', 
    margin='10px 0', 
    border_radius='8px',
    width='95%'
)

def create_header(text):
    return widgets.HTML(f"<div style='{section_header_style}'>{text}</div>")

# Widgets
matlab_widget = widgets.Dropdown(
    options=matlab_candidates,
    value=default_matlab,
    description='MATLAB Run:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='100%')
)

python_widget = widgets.Dropdown(
    options=python_candidates,
    value=default_python,
    description='Python Run:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='100%')
)

output_widget = widgets.Text(
    value=default_output_name,
    description='Output Name:',
    placeholder='Name of the new comparison folder',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='100%')
)


selection_box = widgets.VBox([
    create_header("Select Runs"),
    matlab_widget,
    python_widget,
    create_header("Output"),
    output_widget
], layout=box_layout)

display(selection_box)

## Run Comparison

In [ ]:
run_button = widgets.Button(
    description='Compare Results',
    button_style='primary', # 'success', 'info', 'warning', 'danger' or ''
    layout=widgets.Layout(width='100%', height='50px'),
    icon='balance-scale', 
    style={'font_weight': 'bold', 'font_size': '16px'}
)

output_area = widgets.Output(
    layout=widgets.Layout(
        border='1px solid #ddd',
        padding='10px',
        margin='20px 0',
        max_height='400px',
        overflow='scroll'
    )
)

def on_run_clicked(b):
    # Get values
    matlab_folder = matlab_widget.value
    python_folder = python_widget.value
    output_name = output_widget.value
    
    if not matlab_folder or not python_folder:
        with output_area: print("Error: Please select both runs.")
        return
        
    matlab_path = comparisons_dir / matlab_folder
    python_path = comparisons_dir / python_folder
    output_path = comparisons_dir / output_name

    # Validate folders have content
    if not any((matlab_path / 'matlab_results').iterdir()) if (matlab_path / 'matlab_results').exists() else False:
         with output_area: print(f"Error: Selected MATLAB run {matlab_folder} has no results.")
         return
    if not any((python_path / 'python_results').iterdir()) if (python_path / 'python_results').exists() else False:
         with output_area: print(f"Error: Selected Python run {python_folder} has no results.")
         return

    run_button.disabled = True
    run_button.description = "Comparing..."
    
    with output_area:
        clear_output()
        print(f"Starting comparison at {datetime.now().strftime('%H:%M:%S')}...")
        print("-" * 40)
        
        try:
            status = run_standalone_comparison(
                matlab_dir=matlab_path,
                python_dir=python_path,
                output_dir=output_path,
                project_root=project_root
            )
            
            print("-" * 40)
            if status == 0:
                print("\nComparison report generated successfully!")
            else:
                print("\nComparison failed. Check logs above.")

        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")
            import traceback
            traceback.print_exc()
        finally:
            run_button.disabled = False
            run_button.description = 'Compare Results'

run_button.on_click(on_run_clicked)

display(widgets.VBox([run_button, output_area]))

---
# Visualization

The following sections visualizes the results of the comparison.

In [8]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import json
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add project root to path
project_root = Path('..').resolve()
sys.path.insert(0, str(project_root))

from source.slavv.dev.viz import (
    set_plot_style, 
    plot_count_comparison, 
    plot_radius_distributions
)
from source.slavv.dev.matlab_parser import load_matlab_results_from_output_dir

# Set plot style
set_plot_style()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# Load results for visualization
target_report = None

# If we just ran a comparison (output_path defined), use that
if 'output_path' in locals():
    target_report = output_path / 'comparison_report.json'
    print(f"Using recent run: {output_path.name}")

# Otherwise, try to pick up from the widgets if they exist
elif 'output_widget' in locals() and output_widget.value:
    # Try to construct path from widget value (files might not exist yet if not run)
    target_report = comparisons_dir / output_widget.value / 'comparison_report.json'

if target_report and target_report.exists():
    with open(target_report, 'r') as f:
        comparison = json.load(f)
    print(f"Loaded report successfully.")
else:
    print("No comparison report found. Please run the comparison above first.")
    comparison = None

Loaded report successfully.


## Component Counts

In [10]:
if comparison:
    # Check for empty counts
    m_counts = comparison.get('matlab', {})
    p_counts = comparison.get('python', {})
    total_elements = (
        m_counts.get('vertices_count', 0) + 
        p_counts.get('vertices_count', 0) + 
        m_counts.get('edges_count', 0) + 
        p_counts.get('edges_count', 0)
    )
    
    if total_elements == 0:
        msg = "<div style='background-color: #fff3cd; color: #856404; padding: 15px; border: 1px solid #ffeeba; border-radius: 4px;'><strong>Warning:</strong> The comparison data contains zero elements. Please check that your source runs produced valid output.</div>"
        try:
            from IPython.display import display, HTML
            display(HTML(msg))
        except ImportError:
            print("Warning: The comparison data contains zero elements.")
    else:
        fig = plot_count_comparison(comparison)
        plt.show()

## Radius Distributions

In [11]:
# Load detailed data if available
matlab_radii = []
python_radii = []

# Note: detailed data loading logic typically goes here
# For now using comparison summary if available or basic placeholders

if comparison:
    # Attempt to load Python results from pickles if needed
    # For this dashboard we'll rely on what's in the report or implement loading later
    pass

# Placeholder until full data loading is implemented in notebook
if comparison:
    print("Radius comparison visualization requires loading raw data points.")
    # fig = plot_radius_distributions(comparison, None, None)
    # plt.show()

Radius comparison visualization requires loading raw data points.
